# Comparing students' bets against a pragmatic listener model
Natalia Vélez, February 2022

In [4]:
%matplotlib inline
import sys
import pandas as pd
import numpy as np
from ast import literal_eval

sys.path.append('..')
from utils import int_extract

sys.path.append('../2_behavioral/')
import teaching_models as teach

Load student data:

In [5]:
student_df = pd.read_csv('outputs/student_behavior.csv')
student_df['num_hint'] = (student_df['num_hint'] + 1).astype(int)
student_df['problem'] = student_df['problem'].astype(int)
student_df['teacher'] = student_df.teacher.apply(lambda s: int_extract('(?<=sub-)[0-9]+', s))
student_df['bets'] = student_df.bets.apply(literal_eval)
student_df = student_df.rename(columns={'order': 'trial_order'})

print('Loaded data from %i/140 students' % student_df.worker.nunique())
student_df.head()

Loaded data from 130/140 students


,time_elapsed,teacher,problem,num_trial,num_hint,bets,bonus,worker,trial_order,batch
0,181304,4,34,0.0,1,"[0, 35, 41, 48]",0.013777,ARI1EMAHUA9PO,0,2022-02-02-1303
1,192911,4,34,0.0,2,"[0, 50, 49, 0]",0.020623,ARI1EMAHUA9PO,0,2022-02-02-1303
2,203582,4,34,0.0,3,"[0, 0, 100, 0]",0.041667,ARI1EMAHUA9PO,0,2022-02-02-1303
3,223655,27,15,1.0,1,"[0, 52, 50, 48]",0.014444,ARI1EMAHUA9PO,0,2022-02-02-1303
4,232054,27,15,1.0,2,"[0, 52, 50, 0]",0.021242,ARI1EMAHUA9PO,0,2022-02-02-1303


Merge with teacher examples:

In [45]:
teacher_examples = (
    teach.human_df
    .copy()
    .rename(columns={'subject': 'teacher'})
)
teacher_examples['dummy'] = 1
teacher_examples.head()

# replace ex_idx with cumulative index
teacher_examples['num_hint'] = teacher_examples.groupby(['teacher', 'problem'])['dummy'].cumsum()
teacher_examples = teacher_examples[['teacher', 'problem', 'num_hint', 'example']]

Merge with student data:

In [44]:
student_examples = student_df.merge(teacher_examples)
student_examples = student_examples.sort_values(by=['batch', 'worker', 'time_elapsed']).reset_index(drop=True)
student_examples.head()

,time_elapsed,teacher,problem,num_trial,num_hint,bets,bonus,worker,trial_order,batch,example
0,520211,21,34,0.0,1,"[0, 51, 53, 0]",0.030649,A11F3MA5FWH6SJ,1,2022-02-02-1303,30
1,529123,21,34,0.0,2,"[0, 100, 0, 0]",0.062500,A11F3MA5FWH6SJ,1,2022-02-02-1303,6
2,546683,16,19,1.0,1,"[24, 26, 24, 25]",0.010101,A11F3MA5FWH6SJ,1,2022-02-02-1303,10
3,569395,16,19,1.0,2,"[25, 25, 23, 24]",0.010739,A11F3MA5FWH6SJ,1,2022-02-02-1303,17
4,586563,16,19,1.0,3,"[37, 38, 0, 40]",0.013406,A11F3MA5FWH6SJ,1,2022-02-02-1303,0


Pedagogical model predictions:

In [42]:
model_preds = []
    

,teacher,problem,num_hint,example
0,1,22,1,8
1,1,22,2,27
3,1,18,1,3
4,1,18,2,32
5,1,18,3,7
...,...,...,...,...
3583,30,28,2,20
3584,30,28,3,9
3585,30,13,1,14
3586,30,13,2,20


In [43]:
teacher_examples

,teacher,problem,num_hint,example
0,1,22,1,8
1,1,22,2,27
3,1,18,1,3
4,1,18,2,32
5,1,18,3,7
...,...,...,...,...
3583,30,28,2,20
3584,30,28,3,9
3585,30,13,1,14
3586,30,13,2,20
